In [1]:
from __future__ import division
import os
import sys
import time
import numpy as np
import pybedtools
import pandas as pd
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import MaxNLocator
from matplotlib import rcParams
from matplotlib import colors
from mpltools import style
import mpld3
import matplotlib.gridspec as gridspec
import scipy.spatial.distance as distance
import scipy.cluster.hierarchy as sch
import seaborn as sns

style.use('ggplot')

my_locator = MaxNLocator(6)
rcParams['axes.labelsize'] = 9
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['legend.fontsize'] = 7
rcParams['font.serif'] = ['Computer Modern Roman']
rcParams['text.usetex'] = False
rcParams['figure.figsize'] = 20, 10

/Users/Reza/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/Users/Reza/anaconda/lib/python2.7/site-packages/mpltools/style/__init__.py:42: FutureWarning: 

    The style-sheet functionality in mpltools has been integrated into
    Matplotlib >= 1.4. This module will be removed in a future release.

    Note that style-sheets used by `matplotlib.style` use the standard
    Matplotlib rc-file syntax instead of the INI format used by `mpltools`.
    This mostly means un-quoting strings and changing '=' to ':'.


  """, FutureWarning)
/Users/Reza/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
class targeted(object):

    def __init__(self, list_of_bam_files, region_file, outdir, labels):
        self.list_of_bam_files = list_of_bam_files
        self.region_file = region_file
        self.outdir = outdir
        self.labels = labels
        
    def warning(self):
        N = len(self.list_of_bam_files)
        if N >= 10:
            print u"\u2601" + " You are generating plots for " + str(N) + " bam files, some plots will be really really ugly !"
    
    def get_coverage(self, list_of_bam_files):
        """
        Get coverage from a BAM file,  by looking only inside the regions provided by the region_file
        This function is wrapping the coberageBed from BedTools with no histogram creation

        What to test ?:

        if alignment is empty
        if region file is empty
        if coverage_result is empty

        """
        alignment = pybedtools.BedTool(list_of_bam_files)
        regions = pybedtools.BedTool(self.region_file)
        print 'Calculating coverage over regions ...'
        sys.stdout.flush()
        t0 = time.time()
        coverage_result = alignment.coverage(regions).sort()
        coverage_array = np.array([i[-1] for i in coverage_result], dtype=int)

        t1 = time.time()
        print 'completed in %.2fs' % (t1 - t0)
        sys.stdout.flush()
        return coverage_array

    @staticmethod
    def split_coverage(x):

        if isinstance(x, basestring):
            fn = x
        else:
            fn = x.fn

        f = open(fn)
        hist_lines = []

        def gen():

            while True:
                line = f.next()
                toks = line.strip().split('\t')
                if toks[0] == 'all':
                    # Don't need to include the "all" text in the first field.
                    hist_lines.append(toks[1:])
                    break
                yield pybedtools.create_interval_from_list(toks)

    # Create a BedTool from the generator, and call `saveas` to consume the
    # generator.  We need this so that the file pointer will stop right after
    # the first "all" line.
        b = pybedtools.BedTool(gen()).saveas()

    # Pick up where we left off in the open file, appending to hist_lines.
        while True:
            try:
                line = f.next()
            except StopIteration:
                break
            hist_lines.append(line.strip().split('\t')[1:])

        df = pd.DataFrame(
            hist_lines,
            columns=['depth', 'count', 'size', 'percent'])
        return b, df

    def get_coverage_histogram(self, bam_file, outfile):

        alignment = pybedtools.BedTool(bam_file)
        regions = pybedtools.BedTool(self.region_file)
        coverage = alignment.coverage(regions, hist=True, output=outfile)
        # Now get the coverage and the all histogram
        coverage_histogram, all_histogram = self.split_coverage(coverage)

        for _ in coverage_histogram:
            pass

        return coverage_histogram, all_histogram

    def plot_coverage_heatmap(self, heatmap_name):

        region = pybedtools.BedTool(self.region_file)
        result = region.multi_bam_coverage(bams=self.list_of_bam_files, output=os.path.join(self.outdir, "multicoverage.hist.txt"))
        coverage_df = pd.read_table(result.fn, header=None)
        ncols = coverage_df.shape[1]
        data = coverage_df[list(coverage_df.columns[3:ncols])].astype(int)
        # Set columns
        data.columns = self.labels
        # Set index
        data_index = [str(chrom) + ":" + str(start) + "--" + str(end) for chrom, start, end in zip(list(coverage_df[coverage_df.columns[0]]), list(coverage_df[coverage_df.columns[1]]), list(coverage_df[coverage_df.columns[2]]))]
        data['coordinates'] = data_index
        data = data.set_index('coordinates')
        fig = plt.figure()
        sns.heatmap(data, square=False, annot=True, fmt="d",  annot_kws={"size": 5})
        plt.xticks(rotation=90, fontsize=5)
        plt.yticks(fontsize=5)
        plt.title("Coverage within amplicon regions")
        plt.ylabel("amplicon regions")
        plt.xlabel("samples")
        fig.tight_layout()
        fig.savefig(os.path.join(self.outdir, heatmap_name))

In [8]:
import argparse
import os
from targeted_sequencing_analytics_suite import tsa


parser = argparse.ArgumentParser(description='Process different bam files and output plots on coverage and enrichment')

parser.add_argument('--path_to_bams',
                    help='''Path to bam files, sorted and indexed.''')
parser.add_argument('--targets',
                    help='''File containing amplicon regions in Bed format''')
parser.add_argument('--variant_status_path',
                    help='''Path to binomial test result files (tsv files)''')
parser.add_argument('--bam_extension',
                    help='''Extension of the alignments''')
parser.add_argument('--outdir',
                    help='''directory to save all different plots''')
parser.add_argument('--custom_order', required=False,
                    help='''order of the samples we want to display''' )

args = parser.parse_args()

ImportError: No module named targeted_sequencing_analytics_suite

In [3]:
from pathlib import Path
DATA_DIR = Path('/Users/Reza/GitHub/TruSight_one_BED')

In [6]:
list_of_bam_files=[DATA_DIR/"CL021-336.GATK.recal.bam"]
region_file= DATA_DIR/"TruSight_One_v1.1.bed"
cl021=targeted(list_of_bam_files,
               region_file,
               "/Users/Reza/Github/getTruSight_one_BED",
               "CL021")

In [7]:
cl021.get_coverage(list_of_bam_files)

TypeError: 'PosixPath' object is not iterable